In [1]:
import os
import pandas as pd

In [2]:
# caminho para a pasta onde estão os arquivos CSV
caminho_pasta = 'dados'

# lista para armazenar os dataframes de cada arquivo CSV
dfs = []

In [3]:
# percorre todos os arquivos da pasta com a extensão .csv
for arquivo in os.listdir(caminho_pasta):
    if arquivo.endswith('.csv'):
        # lê o arquivo CSV com o pandas
        df = pd.read_csv(os.path.join(caminho_pasta, arquivo), delimiter=';', skiprows=5, header=5, encoding='ISO-8859-1')
                
        # adiciona as colunas de indicador e equipe
        df_colunas = pd.read_csv(os.path.join(caminho_pasta, arquivo), encoding='ISO-8859-1')
        indicador = df_colunas.iloc[5,0] # valor da linha 7 do arquivo CSV
        indicador = indicador.replace("Indicador: ", "")
        equipe = df_colunas.iloc[7, 0] # valor da linha 9 do arquivo CSV
        df['Indicador'] = indicador
        df['Equipe'] = equipe
        
        # adiciona a coluna com o nome do arquivo
        nome_arquivo = os.path.splitext(arquivo)[0] # remove a extensão do arquivo
        df['Arquivo'] = nome_arquivo

        # Definir Ano
        df['Ano'] = df.columns[5][:4]

        # Renomear coluna de % de quartil
        df = df.rename(columns={df.columns[5]: 'Percentual Quadrimestre'})

        # Definir Quadrimestre
        df['Quadrimestre'] = df['Arquivo'].str[20]
        
        # Remover a coluna vazia
        df = df.drop('Unnamed: 12', axis=1)
        df = df.drop('Arquivo', axis=1)

        # Remover as 5 últimas linas de rosto
        df = df.drop(df.tail(5).index)

        # adiciona o dataframe à lista de dataframes
        dfs.append(df)

In [4]:
# concatena os dataframes em um único dataframe final
df_final = pd.concat(dfs, ignore_index=True)

# Incluir sks
df_final['sk_indicador'] = df_final['Indicador'].map(df_final['Indicador'].drop_duplicates().reset_index(drop=True).reset_index().set_index('Indicador')['index'] + 1)
df_final['sk_equipe'] = pd.factorize(df_final['Equipe'])[0] + 1
df_final['sk_localidade'] = df_final['IBGE'].astype(int)
df_final["sk_periodo"] = df_final["Ano"]+df_final["Quadrimestre"]


In [5]:
# exibe o dataframe final
df_final.head(100000)

,UF,IBGE,Munícipio,Numerador,Denominador Utilizado,Percentual Quadrimestre,Denominador Identificado,Denominador Estimado,Cadastro,Base Externa,Percentual,População,Indicador,Equipe,Ano,Quadrimestre,sk_indicador,sk_equipe,sk_localidade,sk_periodo
0,SP,354700.0,SANTA MARIA DA SERRA,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,6236.0,Proporção de crianças de 1 (um) ano de idade v...,Considerado apenas (eSF e eAP) válidas para o ...,2022,1,1,1,354700,20221
1,MG,316660.0,SERRA DA SAUDADE,0.0,1.0,0.0,0.0,1.0,891.0,1.0,0.0,776.0,Proporção de crianças de 1 (um) ano de idade v...,Considerado apenas (eSF e eAP) válidas para o ...,2022,1,1,1,316660,20221
2,TO,170270.0,AURORA DO TOCANTINS,0.0,10.0,0.0,10.0,6.0,3471.0,6.0,167.0,3783.0,Proporção de crianças de 1 (um) ano de idade v...,Considerado apenas (eSF e eAP) válidas para o ...,2022,1,1,1,170270,20221
3,GO,521940.0,SANTA RITA DO ARAGUAIA,0.0,10.0,0.0,3.0,10.0,4575.0,20.0,30.0,8935.0,Proporção de crianças de 1 (um) ano de idade v...,Considerado apenas (eSF e eAP) válidas para o ...,2022,1,1,1,521940,20221
4,PR,410670.0,CRUZEIRO DO SUL,0.0,11.0,0.0,0.0,11.0,3163.0,15.0,0.0,4449.0,Proporção de crianças de 1 (um) ano de idade v...,Considerado apenas (eSF e eAP) válidas para o ...,2022,1,1,1,410670,20221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,SP,355645.0,VARGEM GRANDE PAULISTA,98.0,197.0,50.0,197.0,226.0,50616.0,242.0,87.0,54315.0,Proporção de gestantes com pelo menos 6 (seis)...,Considerado apenas equipes (eSF e eAP) homolog...,2022,2,2,2,355645,20222
99996,AC,120050.0,SENA MADUREIRA,98.0,255.0,38.0,255.0,195.0,37453.0,246.0,131.0,47168.0,Proporção de gestantes com pelo menos 6 (seis)...,Considerado apenas equipes (eSF e eAP) homolog...,2022,2,2,2,120050,20222
99997,RJ,330220.0,ITAPERUNA,98.0,289.0,34.0,215.0,289.0,86238.0,350.0,74.0,104354.0,Proporção de gestantes com pelo menos 6 (seis)...,Considerado apenas equipes (eSF e eAP) homolog...,2022,2,2,2,330220,20222
99998,PI,220390.0,FLORIANO,98.0,292.0,34.0,292.0,279.0,61203.0,274.0,105.0,60111.0,Proporção de gestantes com pelo menos 6 (seis)...,Considerado apenas equipes (eSF e eAP) homolog...,2022,2,2,2,220390,20222


In [6]:
df_final.columns

Index(['UF', 'IBGE', 'Munícipio', 'Numerador', 'Denominador Utilizado',
       'Percentual Quadrimestre', 'Denominador Identificado',
       'Denominador Estimado', 'Cadastro', 'Base Externa', 'Percentual',
       'População', 'Indicador', 'Equipe', 'Ano', 'Quadrimestre',
       'sk_indicador', 'sk_equipe', 'sk_localidade', 'sk_periodo'],
      dtype='object')

In [7]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233478 entries, 0 to 233477
Data columns (total 20 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   UF                        233478 non-null  object 
 1   IBGE                      233478 non-null  float64
 2   Munícipio                 233478 non-null  object 
 3   Numerador                 233478 non-null  float64
 4   Denominador Utilizado     233478 non-null  float64
 5   Percentual Quadrimestre   233478 non-null  float64
 6   Denominador Identificado  233478 non-null  float64
 7   Denominador Estimado      233478 non-null  float64
 8   Cadastro                  233478 non-null  float64
 9   Base Externa              233478 non-null  float64
 10  Percentual                233478 non-null  float64
 11  População                 233478 non-null  float64
 12  Indicador                 233478 non-null  object 
 13  Equipe                    233478 non-null  o

In [312]:
df_final.shape

(233478, 20)

In [313]:
print('O dataset possui',df_final.shape[0], 'registros e', df_final.shape[1], 'atributos.')

O dataset possui 233478 registros e 20 atributos.


In [314]:
df_final.describe()

,IBGE,Numerador,Denominador Utilizado,Percentual Quadrimestre,Denominador Identificado,Denominador Estimado,Cadastro,Base Externa,Percentual,População,sk_indicador,sk_equipe,sk_localidade
count,233478.000000,233478.000000,2.334780e+05,233478.00000,2.334780e+05,2.334780e+05,2.334780e+05,2.334780e+05,233478.000000,2.334780e+05,233478.000000,233478.000000,233478.000000
mean,325320.624114,557.279825,2.617825e+03,46.03707,2.468784e+03,2.478791e+03,2.899539e+04,1.592412e+03,114.299848,3.819937e+04,4.000000,1.500300,325320.624114
std,98499.244940,3055.431834,2.063719e+04,28.61720,1.919624e+04,1.924772e+04,1.305334e+05,2.488236e+04,59.088050,2.239665e+05,2.000004,0.500001,98499.244940
min,110001.000000,0.000000,0.000000e+00,0.00000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,2.100000e+01,1.000000,1.000000,110001.000000
25%,251207.000000,19.000000,3.500000e+01,21.00000,3.300000e+01,2.900000e+01,5.744000e+03,1.100000e+01,90.000000,5.452000e+03,2.000000,1.000000,251207.000000
50%,314600.000000,71.000000,1.760000e+02,43.00000,1.610000e+02,1.660000e+02,1.121800e+04,2.700000e+01,104.000000,1.169000e+04,4.000000,2.000000,314600.000000
75%,411920.000000,400.000000,1.665000e+03,71.00000,1.607000e+03,1.543000e+03,2.310400e+04,1.340000e+02,126.000000,2.571900e+04,6.000000,2.000000,411920.000000
max,530010.000000,283703.000000,1.928165e+06,100.00000,1.928165e+06,1.704243e+06,5.893815e+06,3.584509e+06,2900.000000,1.239637e+07,7.000000,2.000000,530010.000000


In [315]:
df_final.isna().sum()

UF                          0
IBGE                        0
Munícipio                   0
Numerador                   0
Denominador Utilizado       0
Percentual Quadrimestre     0
Denominador Identificado    0
Denominador Estimado        0
Cadastro                    0
Base Externa                0
Percentual                  0
População                   0
Indicador                   0
Equipe                      0
Ano                         0
Quadrimestre                0
sk_indicador                0
sk_equipe                   0
sk_localidade               0
sk_periodo                  0
dtype: int64

In [316]:
df_indicador = None
df_equipe = None
df_periodo = None
df_localidade = None
fato_sisab = None

DF_INDICADOR

In [317]:
# mapeamento dos valores de de_indicador para nm_indicador
map_indicador = {'Proporção de crianças de 1 (um) ano de idade vacinadas na APS contra Difteria, Tétano, Coqueluche, Hepatite B, infecções causadas por haemophilus influenzae tipo b e Poliomielite inativada': 'criancas_vacinadas',
                 'Proporção de gestantes com pelo menos 6 (seis) consultas pré-natal realizadas, sendo a 1ª (primeira) até a 12ª (décima segunda) semana de gestação': 'gestantes_pre_natal',
                 'Proporção de gestantes com atendimento odontológico realizado': 'gestantes_odontologico',
                 'Proporção de gestantes com realização de exames para sífilis e HIV': 'gestantes_exames_dsts',
                 'Proporção de mulheres com coleta de citopatológico na APS': 'mulheres_citopatologico',
                 'Proporção de pessoas com diabetes, com consulta e hemoglobina glicada solicitada no semestre': 'pessoas_diabetes',
                 'Proporção de pessoas com hipertensão, com consulta e pressão arterial aferida no semestre': 'pessoas_hipertensao'}

In [318]:
df_indicador =  df_final[['sk_indicador','Indicador']].drop_duplicates().reset_index(drop=True)
df_indicador = df_indicador.rename(columns={'Indicador': 'de_indicador'})
df_indicador['nm_indicador'] = df_indicador['de_indicador'].apply(lambda x: map_indicador[x])
df_indicador = df_indicador.reindex(columns=['sk_indicador', 'nm_indicador', 'de_indicador'])

In [319]:
df_indicador.head()

,sk_indicador,nm_indicador,de_indicador
0,1,criancas_vacinadas,Proporção de crianças de 1 (um) ano de idade v...
1,2,gestantes_pre_natal,Proporção de gestantes com pelo menos 6 (seis)...
2,3,gestantes_odontologico,Proporção de gestantes com atendimento odontol...
3,4,gestantes_exames_dsts,Proporção de gestantes com realização de exame...
4,5,mulheres_citopatologico,Proporção de mulheres com coleta de citopatoló...


DF_EQUIPE

In [320]:
# mapeamento dos valores de de_indicador para nm_indicador
map_equipe = {'Considerado apenas (eSF e eAP) válidas para o componente de desempenho': 'validas',
              'Considerado apenas equipes (eSF e eAP) homologadas': 'homologadas'}

In [321]:
df_equipe =  df_final[['sk_equipe','Equipe']].drop_duplicates().reset_index(drop=True)
df_equipe = df_equipe.rename(columns={'Equipe': 'de_equipe'})
df_equipe['nm_equipe'] = df_equipe['de_equipe'].apply(lambda x: map_equipe[x])
df_equipe = df_equipe.reindex(columns=['sk_equipe', 'nm_equipe', 'de_equipe'])

In [335]:
df_equipe.head()

,sk_equipe,nm_equipe,de_equipe
0,1,validas,Considerado apenas (eSF e eAP) válidas para o ...
1,2,homologadas,Considerado apenas equipes (eSF e eAP) homolog...


DF_LOCALIDADE

In [325]:
df_localidade = df_final[['sk_localidade', 'UF', 'Munícipio']].drop_duplicates().reset_index(drop=True)
df_localidade = df_localidade.rename(columns={'UF': 'uf', 'Munícipio': 'municipio'})
#df_localidade.set_index('sk_localidade', inplace=True, drop=False)

In [326]:
df_localidade.head()

,sk_localidade,uf,municipio
0,354700,SP,SANTA MARIA DA SERRA
1,316660,MG,SERRA DA SAUDADE
2,170270,TO,AURORA DO TOCANTINS
3,521940,GO,SANTA RITA DO ARAGUAIA
4,410670,PR,CRUZEIRO DO SUL


DF_PERIODO

In [327]:
df_periodo = df_final[['sk_periodo','Ano','Quadrimestre']].drop_duplicates().reset_index(drop=True)
df_periodo = df_periodo.rename(columns={'Ano': 'ano', 'Quadrimestre': 'quadrimestre'})
df_periodo = df_periodo.reindex(columns=['sk_periodo', 'ano', 'quadrimestre'])

In [328]:
df_periodo.head()

,sk_periodo,ano,quadrimestre
0,20221,2022,1
1,20222,2022,2
2,20223,2022,3


FATO_DESEMPENHO

In [ ]:
# sk_indicador integer NOT NULL,
# sk_equipe integer NOT NULL,
# sk_periodo integer NOT NULL,
# sk_localidade integer NOT NULL,
# numerador integer NOT NULL,
# denominador_utilizado integer NOT NULL,
# percentual_quadrimestre integer NOT NULL,
# denominador_identificado integer NOT NULL,
# denominador_estimado integer NOT NULL,
# cadastro integer NOT NULL,
# base_externa real NOT NULL,
# percentual integer NOT NULL,
# populacao integer NOT NULL,



In [332]:
fato_sisab = df_final[[
    'sk_indicador',
    'sk_equipe',
    'sk_periodo',
    'sk_localidade',
    'Numerador',
    'Denominador Utilizado',
    'Percentual Quadrimestre',
    'Denominador Identificado',
    'Denominador Estimado',
    'Cadastro',
    'Base Externa',
    'Percentual',
    'População'
    ]]

In [333]:
fato_sisab.head()

,sk_indicador,sk_equipe,sk_periodo,sk_localidade,Numerador,Denominador Utilizado,Percentual Quadrimestre,Denominador Identificado,Denominador Estimado,Cadastro,Base Externa,Percentual,População
0,1,1,20221,354700,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,6236.0
1,1,1,20221,316660,0.0,1.0,0.0,0.0,1.0,891.0,1.0,0.0,776.0
2,1,1,20221,170270,0.0,10.0,0.0,10.0,6.0,3471.0,6.0,167.0,3783.0
3,1,1,20221,521940,0.0,10.0,0.0,3.0,10.0,4575.0,20.0,30.0,8935.0
4,1,1,20221,410670,0.0,11.0,0.0,0.0,11.0,3163.0,15.0,0.0,4449.0
